<a href="https://colab.research.google.com/github/jerryjliu/llama_index/blob/main/docs/docs/examples/workflow/router_query_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Router Query Engine

`RouterQueryEngine` chooses the most appropriate query engine from multiple options to process a given query.

This notebook walks through implementation of Router Query Engine, using workflows.

Specifically we will implement [RouterQueryEngine](https://docs.llamaindex.ai/en/stable/examples/query_engine/RouterQueryEngine/).

In [ ]:
!pip install -U llama-index

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "sk-..."

Since workflows are async first, this all runs fine in a notebook. If you were running in your own code, you would want to use `asyncio.run()` to start an async event loop if one isn't already running.

```python
async def main():
    <async code>

if __name__ == "__main__":
    import asyncio
    asyncio.run(main())
```

## Define Events

In [ ]:
from llama_index.core.workflow import Event
from llama_index.core.base.base_selector import SelectorResult
from typing import Dict, List, Any
from llama_index.core.base.response.schema import RESPONSE_TYPE


class QueryEngineSelectionEvent(Event):
    """Result of selecting the query engine tools."""

    selected_query_engines: SelectorResult


class SynthesizeEvent(Event):
    """Event for synthesizing the response from different query engines."""

    result: List[RESPONSE_TYPE]
    selected_query_engines: SelectorResult

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/ravithejad/Desktop/llamaindex/lib/python3.9/sit
[nltk_data]     e-packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Package punkt_tab is already up-to-date!


## The Workflow

`selector:`

1. It takes a StartEvent as input and returns a QueryEngineSelectionEvent.
2. The `LLMSingleSelector`/ `PydanticSingleSelector`/ `PydanticMultiSelector` will select one/ multiple query engine tools.

`generate_responses:`

This function uses the selected query engines to generate responses and returns SynthesizeEvent.

`synthesize_responses:`

This function combines the generated responses and synthesizes the final response if multiple query engines are selected otherwise returns the single generated response.


The steps will use the built-in `StartEvent` and `StopEvent` events.

With our events defined, we can construct our workflow and steps. 

In [ ]:
from llama_index.core.workflow import (
    Context,
    Workflow,
    StartEvent,
    StopEvent,
    step,
)

from llama_index.llms.openai import OpenAI
from llama_index.core.selectors.utils import get_selector_from_llm
from llama_index.core.base.response.schema import (
    PydanticResponse,
    Response,
    StreamingResponse,
)
from llama_index.core.bridge.pydantic import BaseModel
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.schema import QueryBundle
from llama_index.core import Settings

from IPython.display import Markdown, display


class RouterQueryEngineWorkflow(Workflow):
    @step
    async def selector(
        self, ctx: Context, ev: StartEvent
    ) -> QueryEngineSelectionEvent:
        """
        Selects a single/ multiple query engines based on the query.
        """

        ctx.data["query"] = ev.get("query")
        ctx.data["llm"] = ev.get("llm")
        ctx.data["query_engine_tools"] = ev.get("query_engine_tools")
        ctx.data["summarizer"] = ev.get("summarizer")

        llm = Settings.llm
        select_multiple_query_engines = ev.get("select_multi")
        query = ev.get("query")
        query_engine_tools = ev.get("query_engine_tools")

        selector = get_selector_from_llm(
            llm, is_multi=select_multiple_query_engines
        )

        query_engines_metadata = [
            query_engine.metadata for query_engine in query_engine_tools
        ]

        selected_query_engines = await selector.aselect(
            query_engines_metadata, query
        )

        return QueryEngineSelectionEvent(
            selected_query_engines=selected_query_engines
        )

    @step
    async def generate_responses(
        self, ctx: Context, ev: QueryEngineSelectionEvent
    ) -> SynthesizeEvent:
        """Generate the responses from the selected query engines."""

        query = ctx.data.get("query")
        selected_query_engines = ev.selected_query_engines
        query_engine_tools = ctx.data["query_engine_tools"]

        query_engines = [engine.query_engine for engine in query_engine_tools]

        print(
            f"number of selected query engines: {len(selected_query_engines.selections)}"
        )

        if len(selected_query_engines.selections) > 1:
            response_generated = []
            for selected_query_engine in selected_query_engines.selections:
                print(
                    f"Selected query engine: {selected_query_engine.index}: {selected_query_engine.reason}"
                )
                query_engine = query_engines[selected_query_engine.index]
                response_generated.append(query_engine.query(query))

        else:
            query_engine = query_engines[
                selected_query_engines.selections[0].index
            ]

            print(
                f"Selected query engine: {selected_query_engines.ind}: {selected_query_engines.reason}"
            )

            response_generated = [query_engine.query(query)]

        return SynthesizeEvent(
            result=response_generated,
            selected_query_engines=selected_query_engines,
        )

    async def acombine_responses(
        self,
        summarizer: TreeSummarize,
        responses: List[RESPONSE_TYPE],
        query_bundle: QueryBundle,
    ) -> RESPONSE_TYPE:
        """Async combine multiple response from sub-engines."""

        print("Combining responses from multiple query engines.")

        response_strs = []
        source_nodes = []
        for response in responses:
            if isinstance(response, (StreamingResponse, PydanticResponse)):
                response_obj = response.get_response()
            else:
                response_obj = response
            source_nodes.extend(response_obj.source_nodes)
            response_strs.append(str(response))

        summary = await summarizer.aget_response(
            query_bundle.query_str, response_strs
        )

        if isinstance(summary, str):
            return Response(response=summary, source_nodes=source_nodes)
        elif isinstance(summary, BaseModel):
            return PydanticResponse(
                response=summary, source_nodes=source_nodes
            )
        else:
            return StreamingResponse(
                response_gen=summary, source_nodes=source_nodes
            )

    @step
    async def synthesize_responses(
        self, ctx: Context, ev: SynthesizeEvent
    ) -> StopEvent:
        """Synthesizes the responses from the generated responses."""

        response_generated = ev.result
        query = ctx.data.get("query")
        summarizer = ctx.data["summarizer"]
        selected_query_engines = ev.selected_query_engines

        if len(response_generated) > 1:
            response = await self.acombine_responses(
                summarizer, response_generated, QueryBundle(query_str=query)
            )
        else:
            response = response_generated[0]

        response.metadata = response.metadata or {}
        response.metadata["selector_result"] = selected_query_engines

        return StopEvent(result=response)

## Define LLM

In [ ]:
llm = OpenAI(model="gpt-4o-mini")
Settings.llm = llm

## Define Summarizer

In [ ]:
from llama_index.core.prompts.default_prompt_selectors import (
    DEFAULT_TREE_SUMMARIZE_PROMPT_SEL,
)

summarizer = TreeSummarize(
    llm=llm,
    summary_template=DEFAULT_TREE_SUMMARIZE_PROMPT_SEL,
)

## Download Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-08-26 12:24:38--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-08-26 12:24:38 (5.05 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



## Load Data

In [ ]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("./data/paul_graham").load_data()

## Create Nodes

In [ ]:
nodes = Settings.node_parser.get_nodes_from_documents(documents)

## Create Indices

We will create three indices SummaryIndex, VectorStoreIndex and SimpleKeywordTableIndex.

In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SummaryIndex,
    SimpleKeywordTableIndex,
)

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)
keyword_index = SimpleKeywordTableIndex(nodes)

## Create Query Engine Tools

In [ ]:
from llama_index.core.tools import QueryEngineTool

list_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()
keyword_query_engine = keyword_index.as_query_engine()

list_tool = QueryEngineTool.from_defaults(
    query_engine=list_query_engine,
    description=(
        "Useful for summarization questions related to Paul Graham eassy on"
        " What I Worked On."
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from Paul Graham essay on What"
        " I Worked On."
    ),
)

keyword_tool = QueryEngineTool.from_defaults(
    query_engine=keyword_query_engine,
    description=(
        "Useful for retrieving specific context using keywords from Paul"
        " Graham essay on What I Worked On."
    ),
)

query_engine_tools = [list_tool, vector_tool, keyword_tool]

## Run the Workflow!

In [ ]:
import nest_asyncio

nest_asyncio.apply()

w = RouterQueryEngineWorkflow(timeout=200)

### Querying

#### Summarization Query

In [ ]:
# This should use summary query engine/ tool.

query = "Provide the summary of the document?"

result = await w.run(
    query=query,
    llm=llm,
    query_engine_tools=query_engine_tools,
    summarizer=summarizer,
    select_multi=True,  # You can change it to default it to select only one query engine.
)

display(
    Markdown("> Question: {}".format(query)),
    Markdown("Answer: {}".format(result)),
)

number of selected query engines: 1
Selected query engine: 0: This choice directly addresses the need for a summary of the document.


> Question: Provide the summary of the document?

Answer: The document chronicles the journey of an individual who navigated through various fields, including writing, programming, artificial intelligence, and art, before ultimately becoming a prominent figure in the startup ecosystem. It begins with early experiences in writing short stories and programming on an IBM 1401, transitioning to microcomputers and discovering a passion for coding, particularly in Lisp.

The narrative details a shift from studying philosophy to pursuing artificial intelligence, influenced by literature and documentaries. Despite initial enthusiasm, the author became disillusioned with the limitations of AI as practiced at the time, leading to a focus on Lisp and the writing of a book on the subject.

After a stint in art school, the author returned to programming and co-founded Viaweb, an early e-commerce platform that was later acquired by Yahoo. The experience at Viaweb highlighted the importance of growth rates in startups and the challenges of managing a growing company.

Following the acquisition, the author faced a period of burnout but eventually transitioned into angel investing and co-founded Y Combinator, a startup accelerator that revolutionized early-stage funding. The document reflects on the evolution of Y Combinator, its unique batch model, and the supportive community it fostered among startups.

As the narrative progresses, the author contemplates the balance between personal and professional life, particularly after family health issues. The conclusion emphasizes a return to painting and writing essays, alongside the development of a new Lisp dialect called Bel, showcasing a continuous exploration of creativity and technology throughout the author's life.

#### Pointed Context Query

In [ ]:
# This should use vector query engine/ tool.

query = "What did the author do growing up?"

result = await w.run(
    query=query,
    llm=llm,
    query_engine_tools=query_engine_tools,
    summarizer=summarizer,
    select_multi=False,  # You can change it to select multiple query engines.
)

display(
    Markdown("> Question: {}".format(query)),
    Markdown("Answer: {}".format(result)),
)

number of selected query engines: 1
Selected query engine: 1: The question asks for specific context about the author's experiences growing up, which aligns with retrieving specific context from the essay.


> Question: What did the author do growing up?

Answer: Growing up, the author focused on writing and programming outside of school. Initially, he wrote short stories, which he later described as lacking in plot but rich in character emotions. He began programming at a young age on an IBM 1401, using punch cards, but found it puzzling and limited due to the lack of input data. As technology evolved, he transitioned to microcomputers, where he started writing simple games and a word processor. Despite enjoying programming, he initially planned to study philosophy in college, believing it to be a pursuit of ultimate truths. However, he later switched his focus to artificial intelligence, influenced by literature and documentaries.

In [ ]:
# This query could use either a keyword or vector query engine
# so it will combine responses from both

query = "What were noteable events and people from the authors time at Interleaf and YC?"

result = await w.run(
    query=query,
    llm=llm,
    query_engine_tools=query_engine_tools,
    summarizer=summarizer,
    select_multi=True,  # Since query should use two query engine tools, we enabled it.
)

display(
    Markdown("> Question: {}".format(query)),
    Markdown("Answer: {}".format(result)),
)

number of selected query engines: 2
Selected query engine: 1: This choice is useful for retrieving specific context related to notable events and people from the author's time at Interleaf and YC.
Selected query engine: 2: This choice allows for retrieving specific context using keywords, which can help in identifying notable events and people.
Combining responses from multiple query engines.


> Question: What were noteable events and people from the authors time at Interleaf and YC?

Answer: During the author's time at Interleaf, notable events included the existence of a large Release Engineering group, which underscored the complexities of software updates and versioning. The introduction of a scripting language inspired by Emacs, a dialect of Lisp, was also significant. The author reflected on their experience as a bad employee, learning important lessons about the dynamics of technology companies, particularly the value of being run by product people rather than salespeople.

At Y Combinator (YC), key events included a significant deal involving seed funding with Idelle's husband, Julian, which influenced YC's funding model. The transition of leadership to Sam Altman as president marked a pivotal moment, as the author stepped back to ensure YC's evolution and longevity. The author highlighted the contributions of individuals such as Jessica Livingston, Trevor Blackwell, Robert Morris, and the Collison brothers, who played crucial roles in the development of projects and the overall experience at YC. Additionally, the stress of managing Hacker News, which evolved from a news aggregator for startup founders to a broader platform, was a notable aspect of this period.